In [51]:
# Melhorando o modelo 1 com melhores técnicas e usando algoritmos mais otimizados. 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [52]:
caminho_arquivo = r"C:\Users\lipin\OneDrive\Área de Trabalho\Projetos de AI\sao-paulo-properties-april-2019.csv"
df = pd.read_csv(caminho_arquivo)

In [53]:
print(df)

        Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  \
0         930    220    47      2        2       1        1         0   
1        1000    148    45      2        2       1        1         0   
2        1000    100    48      2        2       1        1         0   
3        1000    200    48      2        2       1        1         0   
4        1300    410    55      2        2       1        1         1   
...       ...    ...   ...    ...      ...     ...      ...       ...   
13635  265000    420    51      2        1       0        1         0   
13636  545000    630    74      3        2       1        2         0   
13637  515000   1100   114      3        3       1        1         0   
13638  345000     48    39      1        2       1        1         0   
13639  161987      0    44      2        1       0        1         0   

       Furnished  Swimming Pool  New                 District  \
0              0              0    0    Artur Alvim/São Pa

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13640 entries, 0 to 13639
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             13640 non-null  int64  
 1   Condo             13640 non-null  int64  
 2   Size              13640 non-null  int64  
 3   Rooms             13640 non-null  int64  
 4   Toilets           13640 non-null  int64  
 5   Suites            13640 non-null  int64  
 6   Parking           13640 non-null  int64  
 7   Elevator          13640 non-null  int64  
 8   Furnished         13640 non-null  int64  
 9   Swimming Pool     13640 non-null  int64  
 10  New               13640 non-null  int64  
 11  District          13640 non-null  object 
 12  Negotiation Type  13640 non-null  object 
 13  Property Type     13640 non-null  object 
 14  Latitude          13640 non-null  float64
 15  Longitude         13640 non-null  float64
dtypes: float64(2), int64(11), object(3)
memo

In [55]:
# Criando as novas features(colunas) para melhorar o modelo.
df["Total_de_comodos"] = df["Rooms"] + df ["Toilets"]
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13640 entries, 0 to 13639
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             13640 non-null  int64  
 1   Condo             13640 non-null  int64  
 2   Size              13640 non-null  int64  
 3   Rooms             13640 non-null  int64  
 4   Toilets           13640 non-null  int64  
 5   Suites            13640 non-null  int64  
 6   Parking           13640 non-null  int64  
 7   Elevator          13640 non-null  int64  
 8   Furnished         13640 non-null  int64  
 9   Swimming Pool     13640 non-null  int64  
 10  New               13640 non-null  int64  
 11  District          13640 non-null  object 
 12  Negotiation Type  13640 non-null  object 
 13  Property Type     13640 non-null  object 
 14  Latitude          13640 non-null  float64
 15  Longitude         13640 non-null  float64
 16  Total_de_comodos  13640 non-null  int64 

In [56]:
# Transformando as colunas object em numeros para o modelo poder ler 
df_encoded = pd.get_dummies(df, columns = ["District","Negotiation Type","Property Type"], drop_first=True)

In [57]:
# Ver como ficou o novo DataFrame
print(f"Dimenções do dataframe original:{df.shape}")
print(f"Dimenções do dataframe após o encode:{df_encoded.shape}")

Dimenções do dataframe original:(13640, 17)
Dimenções do dataframe após o encode:(13640, 110)


In [58]:
# Olhando as primeiras linhas para ver as novas colunas
print(df_encoded.head())

   Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  Furnished  \
0    930    220    47      2        2       1        1         0          0   
1   1000    148    45      2        2       1        1         0          0   
2   1000    100    48      2        2       1        1         0          0   
3   1000    200    48      2        2       1        1         0          0   
4   1300    410    55      2        2       1        1         1          0   

   Swimming Pool  ...  District_Vila Leopoldina/São Paulo  \
0              0  ...                               False   
1              0  ...                               False   
2              0  ...                               False   
3              0  ...                               False   
4              0  ...                               False   

   District_Vila Madalena/São Paulo  District_Vila Maria/São Paulo  \
0                             False                          False   
1                 

In [59]:
# Verificando as informações para confirmar que não há mais colunas 'object'
print(df_encoded.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13640 entries, 0 to 13639
Columns: 110 entries, Price to Negotiation Type_sale
dtypes: bool(96), float64(2), int64(12)
memory usage: 2.7 MB
None


In [60]:
# Definindo o 'y', ou seja, a variável que queremos descobrir ou prever. 
y = df_encoded["Price"]
# Se 'y' é "Price" então 'X' são todas as outras colunas que servirão de informação para o modelo.
# Usamos .drop() para criar um DataFrame com todas as colunas, EXCETO a 'Price'.
X = df_encoded.drop("Price", axis=1)
print("Formato das features (X):", X.shape)
print("Formato do alvo (y):", y.shape)

Formato das features (X): (13640, 109)
Formato do alvo (y): (13640,)


In [61]:
# Separando os modelos de treino em 80/20, ou seja, 80% aprendizado e 20% de teste. 
# Definindo 'test_size' automaticamente o restante fica para learn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Tamanho de X para treino:", X_train.shape)
print("Tamanho de X para teste:", X_test.shape)

Tamanho de X para treino: (10912, 109)
Tamanho de X para teste: (2728, 109)


In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

In [63]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [64]:
# Treinando o modelo com o randomforest
modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# 2. Treinar o modelo com os dados de treino
print("Treinando o modelo RandomForest... Isso pode demorar um pouco.")
modelo_rf.fit(X_train, y_train)
print("Modelo treinado com sucesso!")


Treinando o modelo RandomForest... Isso pode demorar um pouco.
Modelo treinado com sucesso!


In [65]:
# Modelos de teste randomforest
previsoes_rf = modelo_rf.predict(X_test)
# Calculando as métricas de avaliação
r2_rf = r2_score(y_test, previsoes_rf)
mae_rf = mean_absolute_error(y_test, previsoes_rf)

print("\n--- Resultados do RandomForestRegressor ---")
print(f"R² (Coeficiente de Determinação): {r2_rf:.2f}")
print(f"MAE (Erro Médio Absoluto): R$ {mae_rf:,.2f}")


--- Resultados do RandomForestRegressor ---
R² (Coeficiente de Determinação): 0.93
MAE (Erro Médio Absoluto): R$ 44,695.03
